# Filesystem tests

## Filesystem methods

In [ ]:
String execCommand(String command) throws Exception {
    // https://www.mkyong.com/java/how-to-execute-shell-command-from-java/

    Process p = Runtime.getRuntime().exec(command);
    p.waitFor();
    BufferedReader r = new BufferedReader(new InputStreamReader(p.getInputStream()));

    StringBuffer sb = new StringBuffer();
    String l = "";
    while ((l = r.readLine()) != null) {
        sb.append(l + "\n");
    }
    return sb.toString();
}

In [ ]:
private static String readFile(String file) throws Exception {
    // https://howtodoinjava.com/java/io/java-read-file-to-string-examples/

    StringBuilder sb = new StringBuilder();
    BufferedReader br = new BufferedReader(new FileReader(file));
    String l;
    while ((l = br.readLine()) != null) {
        sb.append(l).append("\n");
    }
    br.close();
    return sb.toString();
}

## Config: Is current user an admin?

In [ ]:
boolean admin = false;
if("nbgadmin".equals(execCommand("whoami").trim())) {
    admin = true;
}
System.out.println("You are" + (admin ? "" : " not") + " logged in as admin.");

## Tests

In [ ]:
%maven org.junit.jupiter:junit-jupiter-api:5.3.1
import org.junit.jupiter.api.Assertions;

### Home directories

In [ ]:
String user = execCommand("whoami").trim();

for (File f : new File("/home").listFiles()) {
    if(f.isDirectory()) {
        if(f.getName().equals(user) || admin) {
            Assertions.assertTrue(f.canRead(), "canRead " + f.getPath());
        } else {
            Assertions.assertFalse(f.canRead(), "canRead " + f.getPath());
        }
    }
}

### Distribution and exchange directory

In [ ]:
// Read
Assertions.assertEquals("hello world!".trim(), readFile("/srv/distribution/hello-world.txt").trim());

// Write
if(admin) {
    Assertions.assertTrue(new File("/srv/distribution").canWrite(), "canWrite /srv/distribution");
} else {
    Assertions.assertFalse(new File("/srv/distribution").canWrite(), "canWrite /srv/distribution");
}

In [ ]:
// Read
Assertions.assertTrue(new File("/srv/nbgrader/exchange").canRead(), "canRead /srv/nbgrader/exchange");

// Write
if(admin) {
    Assertions.assertTrue(new File("/srv/nbgrader/exchange").canWrite(), "canWrite /srv/nbgrader/exchange");
    Assertions.assertFalse(new File("/srv/nbgrader").canWrite(), "canWrite /srv/nbgrader");
} else {
    Assertions.assertTrue(new File("/srv/nbgrader/exchange").canWrite(), "canWrite /srv/nbgrader/exchange");
    Assertions.assertFalse(new File("/srv/nbgrader").canWrite(), "canWrite /srv/nbgrader");
}

### Configuration directories

In [ ]:
Assertions.assertFalse(new File("/mnt").canRead(), "canWrite /mnt");
Assertions.assertFalse(new File("/mnt").canWrite(), "canWrite /mnt");

File nCfg = new File("/opt/conda/etc/jupyter/nbgrader_config.py");
Assertions.assertFalse(nCfg.canWrite(), "canWrite " + nCfg.getPath());

File jCfg = new File("/srv/jupyterhub/jupyterhub_config.py");
Assertions.assertFalse(jCfg.canWrite(), "canWrite " + jCfg.getPath());

### Installation directories

In [ ]:
for (File f : new File("/opt").listFiles()) {
    
    // Read
    if(f.getName().equals("Oracle_Java") || f.getName().equals("conda")) {
        Assertions.assertTrue(f.canRead(), "canRead " + f.getPath());
    } else {
        Assertions.assertFalse(f.canRead(), "canRead " + f.getPath());   
    }
    
    // Write
    Assertions.assertFalse(f.canWrite(), "canWrite " + f.getPath());
}

In [ ]:
for (File f : new File("/srv").listFiles()) {
    
    // Read
    Assertions.assertTrue(f.canRead(), "canRead " + f.getPath());
    
    // Write
    if(f.getName().equals("distribution")) {
        if(admin) {
            Assertions.assertTrue(f.canWrite(), "canWrite " + f.getPath());
        } else {
            Assertions.assertFalse(f.canWrite(), "canWrite " + f.getPath());
        }
    } else {
        Assertions.assertFalse(f.canWrite(), "canWrite " + f.getPath());
    }
}

## Human tests

In [ ]:
void printFilePermissions(String directory) throws Exception {
    for (File f : new File(directory).listFiles()) {
        System.out.print(f.isDirectory() ? "d" : " ");
        System.out.print(f.canRead() ? "r" : " ");
        System.out.print(f.canWrite() ? "w" : " ");
        System.out.print(f.canExecute() ? "x" : " ");
        System.out.println(" " + f.getCanonicalPath());
    }
}

In [ ]:
System.out.println(execCommand("whoami"));

In [ ]:
printFilePermissions("/home");

In [ ]:
// only if current user equals
printFilePermissions("/home/nbgadmin");

In [ ]:
// only if current user equals or as admin
printFilePermissions("/home/nbguser");

In [ ]:
readFile("/srv/distribution/hello-world.txt");

In [ ]:
// should not work
printFilePermissions("/mnt");

In [ ]:
printFilePermissions("/opt");

In [ ]:
printFilePermissions("/srv");

In [ ]:
printFilePermissions("/srv/nbgrader/");